In [30]:
import pandas as pd

In [31]:
print("Choose input method:")
print("1: Upload a CSV file")
print("2: Enter a Google Sheets link")
choice = input("Enter choice (1 or 2): ")

Choose input method:
1: Upload a CSV file
2: Enter a Google Sheets link
Enter choice (1 or 2): 2


In [32]:
def load_csv():
    from google.colab import files
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    df = pd.read_csv(filename)
    return df

In [33]:
def load_google_sheet(sheet_url):
    try:
        # Extract Google Sheets ID using a more robust method
        import re
        sheet_id = re.search('/spreadsheets/d/([a-zA-Z0-9-_]+)', sheet_url).group(1)

        # Define Google Sheets CSV export URL
        csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"
        df = pd.read_csv(csv_url)
        return df
    except (AttributeError, IndexError):
        # Handle case where the URL format is not as expected.
        print("Error: Invalid Google Sheets URL format.")
        return None

In [34]:
if choice == '1':
    df = load_csv()
elif choice == '2':
    sheet_url = input("Enter the Google Sheets link: ")
    df = load_google_sheet(sheet_url)
else:
    print("Invalid choice.")
    df = None

Enter the Google Sheets link: https://docs.google.com/spreadsheets/d/1wz3iIdlipf2GA1HcAbsX-kHjz01iRnxuJgzTRk87J3w/edit?gid=255587569#gid=255587569


https://docs.google.com/spreadsheets/d/1wz3iIdlipf2GA1HcAbsX-kHjz01iRnxuJgzTRk87J3w/edit?usp=sharing


In [35]:
if df is not None:
    print("First few rows of the dataset:")
    print(df.head())

if df is not None:
    print("\nDataset Information:")
    print(df.info())

First few rows of the dataset:
   Serial No.                                           Airports  \
0           1    Chhatrapati Shivaji International Airport – BOM   
1           2             Kempegowda International Airport – BLR   
2           3                Chennai International Airport – MAA   
3           4  Netaji Subhas Chandra Bose International Airpo...   
4           5  Chaudhary Charan Singh International Airport –...   

      General To                   Type    Latitude  Latitude_1   Longitude  \
0  Coastal plain  International Airport   19 5' 23"    19.09894   72 52' 5"   
1        Plateau  International Airport  13 11' 55"    13.19923  77 42' 24"   
2  Coastal plain  International Airport  12 59' 39"    12.98242   80 10' 9"   
3  Coastal plain  International Airport  22 39' 17"    22.65395  88 26' 48"   
4          plain  International Airport  26 45' 38"    26.76568  80 53' 10"   

   Longitude_1  Altitudes        Role  
0     72.87516         11  Commercial  
1    

In [36]:
print("\nAvailable columns:")
for i, col in enumerate(df.columns):
    print(f"{i + 1}: {col}")

# Prompt user to select a column
col_choice = int(input("Select a column by number: ")) - 1

# Store the selected column name in `selected_attribute`
if 0 <= col_choice < len(df.columns):
    selected_attribute = df.columns[col_choice]
    print(f"\nYou selected: {selected_attribute}")
else:
    print("Invalid column selection.")


Available columns:
1: Serial No.
2: Airports
3: General To
4: Type
5: Latitude
6: Latitude_1
7: Longitude
8: Longitude_1
9: Altitudes
10: Role
Select a column by number: 2

You selected: Airports


In [37]:
if selected_attribute:
    prompt = input("Enter a prompt, Note: You should enclosed the value of column in curly brackets{}. For example 'Give me the contact details of {Chhatrapati Shivaji International Airport – BOM}' where you have selected the Airports field")

Enter a prompt, Note: You should enclosed the value of column in curly brackets{}. For example 'Give me the contact details of {Chhatrapati Shivaji International Airport – BOM}' where you have selected the Airports fieldI want to get details of {Chhatrapati Shivaji International Airport – BOM}


In [38]:
print(prompt)
!pip install google-search-results

I want to get details of {Chhatrapati Shivaji International Airport – BOM}


In [39]:
serpApi = "40fe77b8cd91fac2fd10b3abccfe946a9f70dc2a72f2ed6888ab1fd587f745e2"

In [40]:
import re
from serpapi import GoogleSearch

match = re.search(r'\{(.+?)\}', prompt)
selected_value = match.group(1)

def searchGoogle(prompt):
    """
    Function to search Google using SerpAPI and return search results.
    """
    # match = re.search(r'\{(.+?)\}', prompt)
    if not match:
        return "No value found in curly braces in the prompt."

    # selected_value = match.group(1)
    search_query = prompt.replace(f"For {{{selected_value}}}", str(df[selected_attribute].iloc[0]))
    print(f"Search Query: {search_query}")

    params = {
        "engine": "google",
        "q": search_query,
        "api_key": serpApi
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    if "organic_results" not in results or not results["organic_results"]:
        return "No search results found."

    # Filter and format results
    formatted_results = []
    for result in results.get("organic_results", []):
        title = result.get("title", "No Title")
        link = result.get("link", "No Link")
        snippet = result.get("snippet", "No Snippet")
        formatted_results.append({
            "title": title.strip(),
            "link": link.strip(),
            "snippet": snippet.strip()
        })

    return formatted_results


In [41]:
import requests

GroqAPi = "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"

In [42]:
!pip install groq

In [43]:
from groq import Groq

# Initialize Groq client
client = Groq(api_key=GroqAPi)

# Initialize chat history
chat_history = [
    {
        "role": "system",
        "content": "You are an assistant specialized in extracting specific information from web pages based on a query. Provide only the requested information. If no relevant data is available, respond with 'Result not found' only and only."
    }
]


mapping = {}


def getResult(results, backPrompt, chat_history):
    """
    Extract and present information in a structured and concise format.
    """
    MAX_HISTORY_LENGTH = 5
    chat_history = chat_history[-MAX_HISTORY_LENGTH:]

    all_responses = []
    organic_results = results

    for batch in organic_results:
        # Formulate user query for Groq
        chat_history.append({
            "role": "user",
            "content": f"""
            I have a search result. Please extract the requested details:

            Search Result:
            Title: {batch['title']}
            Link: {batch['link']}
            Snippet: {batch['snippet']}

            Extraction Task:
            {backPrompt}

            Provide only the requested information.
            """
        })

        # Send request to Groq
        completion = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=chat_history
        )

        ai_response = completion.choices[0].message.content
        chat_history.append({
            "role": "assistant",
            "content": ai_response
        })

        # Consolidate responses
        if "result not found" not in ai_response.lower():
            all_responses.append(ai_response)

    if not all_responses:
        return "Result not found."

    # Format responses into a single output
    # return "\n".join(all_responses)
    return combineResponses(all_responses)


def combineResponses(responses):
    """
    Dynamically aggregate and deduplicate responses into a single coherent response.
    """
    unique_responses = set(responses)  # Remove duplicates
    combined_response = "\n".join(unique_responses)

    # Additional formatting (if required)
    result = f"**Extracted Information:**\n{combined_response}"
    return result.strip()


while True:
    backPrompt = input(f"Enter the details you want to fetch about the selected attribute: ('0' to end) ")
    if backPrompt == '0':
        break

    search_results = searchGoogle(prompt)
    aiResponse = getResult(search_results, backPrompt, chat_history)
    print("Extracted Information:")
    print(aiResponse)
    print('\n\n')
    mapping[backPrompt] = aiResponse
    print(mapping)
    print('\n\n')


Enter the details you want to fetch about the selected attribute: ('0' to end) I want to get information desk contact details
Search Query: I want to get details of {Chhatrapati Shivaji International Airport – BOM}
Extracted Information:
**Extracted Information:**
Contact details for the information desk:

* Information available on the official website: None found
Contact information for the Information Desk:

Phone: 022 66851010
Contact information for the Information Desk:

Phone: +91-22-6685 1031

Note: Phone number may be subject to change, it's recommended to check with the airport's official website or other reliable sources for the most up-to-date information.
Information Desk Contact Details:

Phone: +91-22-6685-1031, +91-22-66851234 (available 24 hours)

Note: Phone numbers may be subject to change, it's recommended to check with the airport's official website or other reliable sources for the most up-to-date information.



{'I want to get information desk contact details': 

In [45]:
import pandas as pd
from google.colab import files

# File path for the results file
file_path = "results.csv"

# Prepare data for appending
data = {"User Query": [], "AI Response": []}
for query, response in mapping.items():
    data["User Query"].append(query)
    data["AI Response"].append(response)

# Convert mapping to DataFrame
df = pd.DataFrame(data)

try:
    # Append to existing file if it exists
    existing_df = pd.read_csv(file_path)
    updated_df = pd.concat([existing_df, df], ignore_index=True)
    updated_df.to_csv(file_path, index=False)
except FileNotFoundError:
    # Create new file if it doesn't exist
    df.to_csv(file_path, index=False)

print(f"Results file created or updated at {file_path}.")

# Ask for confirmation before downloading
download_confirmation = input("Are you satisfied with the results and want to download the file? (yes/no): ")

if download_confirmation.lower() == "yes":
    files.download('results.csv')
else:
    print("Download cancelled.")

Results file created or updated at results.csv.
Are you satisfied with the results and want to download the file? (yes/no): yes


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>